In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from talib import abstract
import math
## This is to Convert a string representation of list into list 
import ast
from functools import partial
import os
from ta.momentum import rsi
from ta.utils import dropna
# import pandas_ta as pta
from collections import defaultdict


In [2]:
# files='importdatafiles/01-07--OCT-2021-WEEKLY-expiry_data__VEGE_NF_AND_BNF_Options.csv'

In [37]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)

In [3]:
def resampler(df,time):
    newdf=pd.DataFrame()
    newdf['Open']=df.Open.resample(time).first()
    newdf['Close']=df.Close.resample(time).last()
    newdf['High']=df.High.resample(time).max()
    newdf['Low']=df.Low.resample(time).min()
    newdf['Tick_Volume']=df['Tick Volume'].resample(time).sum()
    newdf['Volume']=df.Volume.resample(time).sum()
    newdf.dropna(axis=0,inplace=True)
    
    return newdf

In [5]:
df=pd.read_csv(files)
df['Date/Time']=pd.to_datetime(df['Date/Time'].astype(str), format='%d-%m-%Y %H:%M:%S')
df.rename(columns={'Date/Time':'Date','Volume':'Tick Volume','Open Interest':'Volume'},inplace=True)

In [89]:
datadf=df.loc[df.Ticker=='NIFTY']
datadf.set_index('Date',inplace=True)
datadf=datadf[-300:]
datadf5=resampler(datadf,'5min')
datadf5=datadf5[-200:].copy()
datadf15=resampler(datadf,'15min')
datadf15=datadf15[-200:].copy()

In [90]:
supertrendBTF=Supertrend(datadf5,10,1)
supertrendBTF['rsi']=rsi(datadf5.Close,window=8,fillna=True)
supertrendBTF['rsima']=supertrendBTF['rsi'].rolling(13).mean()
datadf5=datadf5.join(supertrendBTF)
datadf5.reset_index(inplace=True)

In [92]:
datadf5.head()

,Date,Open,Close,High,Low,Tick_Volume,Volume,Supertrend,Final Lowerband,Final Upperband,rsi,rsima
0,2021-10-07 10:30:00,17791.9,17809.1,17809.5,17791.9,4,0,True,NaN,NaN,100.000000,NaN
1,2021-10-07 10:35:00,17809.0,17817.0,17817.6,17806.3,5,0,True,NaN,NaN,100.000000,NaN
2,2021-10-07 10:40:00,17816.6,17816.6,17820.2,17810.5,5,0,True,NaN,NaN,94.529915,NaN
3,2021-10-07 10:45:00,17816.8,17808.7,17819.7,17806.3,5,0,True,NaN,NaN,42.301388,NaN
4,2021-10-07 10:50:00,17809.0,17808.3,17809.1,17801.0,5,0,True,NaN,NaN,40.990848,NaN


In [83]:
supertrendHTF=Supertrend(datadf15,10,1)
supertrendHTF['rsi']=rsi(datadf15.Close,window=8,fillna=True)
supertrendHTF['rsima']=supertrendHTF['rsi'].rolling(13).mean()
datadf15=datadf15.join(supertrendHTF)
datadf15.reset_index(inplace=True)

In [96]:
datadf=datadf.reset_index()

In [93]:
datadf5_15=pd.merge(datadf5,datadf15,on='Date',how='left')
datadf5_15=datadf5_15.replace(to_replace=np.nan,method='ffill')

In [98]:
maindf=pd.merge(datadf,datadf5_15,on='Date',how='left')

In [99]:
maindf=maindf.replace(to_replace=np.nan,method='ffill')

In [101]:
maindf[-20:]

,Date,Ticker,Open,High,Low,Close,Tick Volume,Volume,Open_x,Close_x,...,Final Lowerband,Final Upperband,rsi,rsima,Open_y,Close_y,High_y,Low_y,Tick_Volume_y,Volume_y
280,2021-10-07 15:11:00,NIFTY,17791.8,17793.2,17790.3,17791.3,1,0,17796.0,17790.3,...,17834.634025,17807.384762,28.362009,35.896452,17809.5,17790.3,17810.3,17787.4,15.0,0.0
281,2021-10-07 15:12:00,NIFTY,17791.2,17794.1,17789.0,17793.7,1,0,17796.0,17790.3,...,17834.634025,17807.384762,28.362009,35.896452,17809.5,17790.3,17810.3,17787.4,15.0,0.0
282,2021-10-07 15:13:00,NIFTY,17793.9,17793.9,17789.5,17790.5,1,0,17796.0,17790.3,...,17834.634025,17807.384762,28.362009,35.896452,17809.5,17790.3,17810.3,17787.4,15.0,0.0
283,2021-10-07 15:14:00,NIFTY,17790.5,17791.0,17789.0,17790.3,1,0,17796.0,17790.3,...,17834.634025,17807.384762,28.362009,35.896452,17809.5,17790.3,17810.3,17787.4,15.0,0.0
284,2021-10-07 15:15:00,NIFTY,17790.8,17791.3,17785.8,17788.3,1,0,17790.8,17788.8,...,17834.634025,17802.134288,27.522567,33.537199,17790.8,17797.2,17801.4,17780.5,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2021-10-07 15:26:00,NIFTY,17790.5,17795.0,17789.2,17794.1,1,0,17788.5,17797.2,...,17834.634025,17798.124942,40.622522,31.785229,17790.8,17797.2,17801.4,17780.5,15.0,0.0
296,2021-10-07 15:27:00,NIFTY,17795.0,17796.1,17793.8,17795.3,1,0,17788.5,17797.2,...,17834.634025,17798.124942,40.622522,31.785229,17790.8,17797.2,17801.4,17780.5,15.0,0.0
297,2021-10-07 15:28:00,NIFTY,17795.8,17798.6,17794.5,17796.3,1,0,17788.5,17797.2,...,17834.634025,17798.124942,40.622522,31.785229,17790.8,17797.2,17801.4,17780.5,15.0,0.0
298,2021-10-07 15:29:00,NIFTY,17796.4,17801.4,17795.3,17797.2,1,0,17788.5,17797.2,...,17834.634025,17798.124942,40.622522,31.785229,17790.8,17797.2,17801.4,17780.5,15.0,0.0


In [ ]:
## def predf(data):
    
    ###Base Time Frame########
    datadf=data.copy()
    datadf.set_index('Date',inplace=True)
    datadf=resampler(datadf,'5min')
    supertrendBTF=Supertrend(datadf,10,1)
    supertrendBTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendBTF['rsima']=supertrendBTF['rsi'].rolling(13).mean()
    supertrendBTF.reset_index(inplace=True)

    ###High Time Frame#########
    datadf=data.copy()
    datadf.set_index('Date',inplace=True)
    datadf=resampler(datadf,'15min')
    supertrendHTF=Supertrend(datadf,10,0.8)
    supertrendHTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendHTF['rsima']=supertrendHTF['rsi'].rolling(13).mean()
    supertrendHTF.reset_index(inplace=True)
    
    #######Merge Base and HTF indicators###
    st=pd.merge(supertrendBTF,supertrendHTF,on='Date',how='left')
    st=st.replace(to_replace=np.nan,method='ffill')
    st=st.set_index('Date')
    
    #########Join everything in single df#######
    datadf=data.copy()
    datadf.set_index('Date',inplace=True)
    datadf=resampler(datadf,'5min')
    datadf=datadf.join(st)
    # print(datadf.head())
    
    return datadf

In [4]:
buy_script=''

In [2]:
buy_script

'ABC'

In [5]:
if buy_script:
    print (buy_script)
else:
    print ('no script')

no script


In [6]:
import time

In [13]:
time.time()

1647249631.7825234

In [15]:
# Get the epoch
obj = time.gmtime(0)
epoch = time.asctime(obj)
print("epoch is:", epoch)

epoch is: Thu Jan  1 00:00:00 1970


In [21]:
datetime.time(15,0,1)

datetime.time(15, 0, 1)

In [53]:
float(str(datetime.datetime.now().time()).split(':')[2])

55.435918

In [33]:
str(datetime.datetime.now().time()).

'15:04:43.720940'

In [54]:
now=datetime.datetime.now().time()


In [55]:
now

datetime.time(17, 33, 42, 478882)

In [61]:
now >= datetime.time(17,35)

False

In [72]:
str(datetime.datetime.now().date())

'2022-03-14'